# Project III: Classical Non-linear Models and Police use of force
This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import probit as probit
import logit as logit
import estimation as est


### Load the data 

In [2]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

In [3]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sage']
dat['sage'] = dat['sage'] / 10

#['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior', 'year']  # Dropped 'osplit' and year // Multikollinearitet mellem hhv. "omajblack", "omajhisp" "omajwhite" og "omajother" +  "sblack", "shisp", "swhite", "sother"

## Declare labels, her er sother og omajother taget ud, men I kan bare rette til white
#y_lab = 'anyuseofforce_coded'
#x_lab = ["const", "sblack", "shisp", "swhite",  # Drop 'sother' giver Multikollinearitet  sammen med "sblack", "shisp", "swhite"
#         "smale", "sage", "sempl", "sincome", 
#         "spop", "daytime", "inctype_lin", 
#         "omajblack", "omajhisp", "omajwhite",  # Drop 'omajother' giver Multikollinearitet sammen med "omajblack", "omajhisp", "omajwhite"
#         "sbehavior"] # dropped year and osplit (indeholder kun 0?)

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset without 'osplit'
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.head(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sage
0,0,1.0,1,0,0,1,1.8
1,0,1.0,1,0,0,1,2.0
2,0,1.0,1,0,0,1,2.2
3,0,1.0,1,0,0,1,2.2
4,0,1.0,1,0,0,1,2.2


In [12]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

# Correlation matrix to check multicollinearity
corr_matrix = pd.DataFrame(dat[x_lab]).corr()
print("Updated Correlation Matrix:\n", corr_matrix)



Updated Correlation Matrix:
         const    sblack     shisp    sother     smale      sage
const     NaN       NaN       NaN       NaN       NaN       NaN
sblack    NaN  1.000000 -0.118565 -0.079767 -0.007463 -0.039889
shisp     NaN -0.118565  1.000000 -0.076088  0.037650 -0.099664
sother    NaN -0.079767 -0.076088  1.000000  0.014754 -0.044000
smale     NaN -0.007463  0.037650  0.014754  1.000000 -0.018036
sage      NaN -0.039889 -0.099664 -0.044000 -0.018036  1.000000


## Estimate using Probit

In [ ]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Display starting values
print("Starting values for theta:", theta0)

probit_results = est.estimate(probit.q, theta0, y, x)

Starting values for theta: [ 0.01950952  0.00861707  0.02755749  0.00319188  0.01218717 -0.00423526]
Optimization terminated successfully.
         Current function value: 0.028810
         Iterations: 48
         Function evaluations: 392
         Gradient evaluations: 56


In [7]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeded after 48 iter. (392 func. evals.). Final criterion:  0.02881.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.4026,0.3058,-7.8562
sblack,0.2567,0.2538,1.0112
shisp,0.4965,0.2040,2.4330
sother,0.1582,0.4338,0.3645
smale,0.4176,0.2230,1.8723
sage,-0.1635,0.0980,-1.6682


## Estimate using Logit

In [8]:
logit_results = est.estimate(logit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.028891
         Iterations: 78
         Function evaluations: 553
         Gradient evaluations: 79


In [10]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeded after 78 iter. (553 func. evals.). Final criterion:  0.02889.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-4.8380,0.8876,-5.4509
sblack,0.7342,0.6865,1.0695
shisp,1.3213,0.5357,2.4667
sother,0.3257,1.3299,0.2449
smale,1.1300,0.6258,1.8057
sage,-0.4465,0.2879,-1.5505


## Partial and Average Effects

## Standard errors of the marginal effects with the Delta Method
